# Single Fidelity Bayesian Optimization

For the model we are using:
- [SingleTaskGP](https://botorch.org/api/_modules/botorch/models/gp_regression.html#SingleTaskGP)
- [RBFKernel](https://docs.gpytorch.ai/en/latest/kernels.html#gpytorch.kernels.RBFKernel)
- [ExactMarginalLogLikelihood]()
- [fit_gpytorch_model](https://botorch.org/api/_modules/botorch/fit.html#fit_gpytorch_model)
- [fit_gpytorch_torch](https://botorch.org/api/_modules/botorch/optim/fit.html#fit_gpytorch_torch)
- [ExpectedImprovement](https://botorch.org/api/_modules/botorch/acquisition/analytic.html#ExpectedImprovement)

In [1]:
import torch
import gpytorch

from botorch.models import SingleTaskGP
from gpytorch.kernels.rbf_kernel import RBFKernel

from botorch.acquisition.analytic import ExpectedImprovement
from botorch.models.transforms.outcome import Standardize

from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
###
#  fix Cholecky jitter error:
#  the line search in the L-BFGS algorithm, used by default, can take some very large steps, 
#  which in turn causes numerical issues in the solves in the underlying gpytorch model.
#
#  recommended solution: use an optimizer from torch.optim to fit the model
#  see [posted issue here](https://github.com/pytorch/botorch/issues/179#issuecomment-504798767)
###
from botorch.optim.fit import fit_gpytorch_torch 

from scipy.stats import norm
import numpy as np

import h5py
import pickle
import os
import time

## Load Data

In [2]:
###
#  load data: targets and features
###
normalization = "normalized" 
file = h5py.File("./targets_and_{}_features.jld2".format(normalization), "r")

# feature matrix
X = torch.from_numpy(np.transpose(file["X"][:]))
# simulation data (high fidelity)
y = torch.from_numpy(np.transpose(file["gcmc_y"][:]))
# associated simulation costs
cost = torch.from_numpy(np.transpose(file["gcmc_elapsed_time"][:]))

# total number of COFs in data set
nb_COFs = X.shape[0]

In [3]:
###
#  load data: initializing COFs
###
init_cof_ids_file = pickle.load(
                    open('../search_results/{}/initializing_cof_ids_{}.pkl'.format(normalization, normalization), 
                         'rb'))

init_cof_ids = init_cof_ids_file['init_cof_ids']

# total number of BO searches to run = number of initializing sets
nb_runs = len(init_cof_ids)

###
#  print info
###
print("raw data - \n\tX:", X.shape)
print("\t\ty:", y.shape)
print("\t\tcost: ", cost.shape)    
    
print("\nEnsure features are normalized - ")
print("max:\n", torch.max(X, 0).values)
print("min:\n", torch.min(X, 0).values)
print("width:\n",torch.max(X, 0).values - torch.min(X, 0).values)

raw data - 
	X: torch.Size([608, 14])
		y: torch.Size([608])
		cost:  torch.Size([608])

Ensure features are normalized - 
max:
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)
min:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)
width:
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)


In [4]:
X_unsqueezed = X.unsqueeze(1)

## Helper Functions

#### Analysis

In [5]:
# accumulated cost lags the index of the cost_acquired (iteration COF is identified) by 1
def accumulated_cost(cost_acquired):
    nb_iters = len(cost_acquired)
    accumulated_cost = np.zeros(nb_iters)
    accumulated_cost[0] = cost_acquired[0]
    for i in range(1, len(cost_acquired)):
        accumulated_cost[i] = accumulated_cost[i-1] + cost_acquired[i]
    return accumulated_cost

In [6]:
def get_y_maxes_acquired(y_acquired):
    nb_iters = len(y_acquired)
    return [max(y_acquired[:i+1]) for i in range(nb_iters)]      

#### Construct Initial Inputs

In [7]:
# construct feature matrix of acquired points
def build_X_train(ids_acquired):
    return X[ids_acquired, :]

# construct output vector for acquired points
def build_y_train(ids_acquired):
    return y[ids_acquired].unsqueeze(-1)

# construct vector to track accumulated cost of acquired points
def build_cost(ids_acquired):
    return cost[ids_acquired]

In [8]:
# construct and fit GP model
def construct_and_fit_gp_model(X_train, y_train):      
    model = SingleTaskGP(X_train, y_train, outcome_transform=Standardize(m=1), covar_module=RBFKernel())
    mll   = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll, optimizer=fit_gpytorch_torch)
    return model

#### Bayesian Algorithm

In [9]:
def run_Bayesian_optimization(nb_iterations, initializing_COFs, verbose=False):
    assert nb_iterations > len(initializing_COFs)
    ###
    #  initialize system
    ###
    # select initial COFs for training data
    ids_acquired = initializing_COFs
    
    ###
    #  itterate through remaining budget using BO
    ###
    for i in range(len(initializing_COFs), nb_iterations):
        # construct training data (perform experiments)
        X_train = build_X_train(ids_acquired)
        y_train = build_y_train(ids_acquired)
        
        # fit GP surrogate model
        model = construct_and_fit_gp_model(X_train, y_train)
        
        # compute expected improvement acquisition function
        acq_fn   = ExpectedImprovement(model, best_f=y_train.max().item())
        acq_vals = acq_fn.forward(X.unsqueeze(1))
        
        # identify COF with highest acquisition value currently *not* in training set
        ids_sorted_by_aquisition = acq_vals.argsort(descending=True)
        for id_max in ids_sorted_by_aquisition:
            if not id_max.item() in ids_acquired:
                id_max_acq = id_max.item()
                break
                
        ###
        #  acquire this COF
        ###
        ids_acquired = np.concatenate((ids_acquired, [id_max_acq]))
        if verbose:
            print("BO iteration ", i)
            print("\tacquired COF ", id_max_acq)
            print("\ty = ", y[id_max_acq].item())
    
    # check budget constraint is stisfied
    assert np.size(ids_acquired) == nb_iterations
    assert len(np.unique(ids_acquired)) == nb_iterations
    return ids_acquired

# Run BO

In [10]:
nb_iterations = 150
nb_COFs_initialization = 3

In [11]:
for i, initializing_COFs in enumerate(init_cof_ids): 
    # check the length of each initializing set
    assert len(initializing_COFs) == nb_COFs_initialization
    print("run #: {}".format(i))
    
    # start timer for BO run
    start_time    = time.time()
    ###
    #  run BO search
    ###
    ids_acquired  = run_Bayesian_optimization(nb_iterations, initializing_COFs, verbose=True)
    
    ###
    #  post-run analysis
    ###
    elapsed_time  = time.time() - start_time
    y_acquired    = y[ids_acquired]
    y_maxes_acq   = get_y_maxes_acquired(y_acquired.detach().numpy())
    cost_acquired = build_cost(ids_acquired)
    acc_cost      = accumulated_cost(cost_acquired.detach().numpy())
        
    BO_iter_top_cof_acquired = np.argmax(y_acquired == max(y))
    print("iteration we acquire top COF = ", BO_iter_top_cof_acquired.item())
    
    top_cof_acc_cost = sum(cost_acquired[:BO_iter_top_cof_acquired]).item() 
    print("accumulated cost up to observation of top COF = ", top_cof_acc_cost, " [min]")
    
    ###
    #  Store SFBO results
    ###
    sfbo_res = dict({'ids_acquired': ids_acquired,
                     'y_acquired': y_acquired.detach().numpy(),
                     'y_max_acquired': y_maxes_acq,
                     'cost_acquired': cost_acquired.detach().numpy(),
                     'accumulated_cost': acc_cost / 60,
                     'nb_COFs_initialization': nb_COFs_initialization,
                     'BO_iter_top_cof_acquired': BO_iter_top_cof_acquired.item(),
                     'elapsed_time (min)':  elapsed_time / 60
                    })

    with open('../search_results/{}/sfbo_results/sfbo_results_run_{}.pkl'.format(normalization, i), 'wb') as file:
        pickle.dump(sfbo_res, file)

# Random Search

In [12]:
nb_rs_runs = 1000
top_cof_id = np.argmax(y).item()

rs_res = dict()
rs_res['ids_acquired']     = []
rs_res['cost_acquired']    = []
rs_res['accumulated_cost'] = []

for r in range(nb_rs_runs):
    rs_ids_acquired     = np.random.choice(range(nb_COFs), replace=False, size=nb_COFs) #  size=nb_iterations
    rs_cost_acquired    = build_cost(rs_ids_acquired)
    rs_accumulated_cost = accumulated_cost(rs_cost_acquired)
    rs_res['ids_acquired'].append(rs_ids_acquired)
    rs_res['cost_acquired'].append(rs_cost_acquired)
    rs_res['accumulated_cost'].append(rs_accumulated_cost / 60) # unit: hours


In [13]:
###
#  test for duplicate runs
###
def check_random_for_dupicates(rs_res):
    for r in range(nb_rs_runs -1):
        for rs_ids_acquired in rs_res['ids_acquired'][r+1:]:
            if all(rs_res['ids_acquired'][r] == rs_ids_acquired):
                print("duplicate run found!")
                return True
    return False

check_random_for_dupicates(rs_res)

False

In [15]:
def test_rs(rs_res):
    for a in np.random.choice(nb_rs_runs, replace=False, size=7):
        # check that the lengths 
        assert len(rs_res['ids_acquired'][a]) == nb_COFs
        assert all([cof_id in range(nb_COFs) for cof_id in rs_res['ids_acquired'][a]])
    return

test_rs(rs_res)

In [16]:
# get y_max acquired up to iteration i for i = 1,2,...
def y_max(rs_ids_acquired):
    y_max_mu      = np.zeros(nb_iterations)
    y_max_sig_bot = np.zeros(nb_iterations)
    y_max_sig_top = np.zeros(nb_iterations)
    # element i of these will be y max at BO iter i
    
    for i in range(1, nb_iterations+1):
        # max value acquired up to this point
        y_maxes = np.array([max(y[rs_ids_acquired[r][:i]]) for r in range(nb_rs_runs)])
        assert np.size(y_maxes) == nb_rs_runs
        y_max_mu[i-1]      = np.mean(y_maxes)
        y_max_sig_bot[i-1] = np.std(y_maxes[y_maxes < y_max_mu[i-1]])
        y_max_sig_top[i-1] = np.std(y_maxes[y_maxes > y_max_mu[i-1]])
    return y_max_mu, y_max_sig_bot, y_max_sig_top

# rs_mean, rs_lower_bound, rs_upper_bound = y_max(rs_res)
y_rs_max_mu, y_rs_max_sig_bot, y_rs_max_sig_top = y_max(rs_res['ids_acquired'])

In [17]:
###
#  Store Random Search Results
###
random_search_results = dict({'ids_acquired': rs_res['ids_acquired'],
                             'y_rs_max_mu': y_rs_max_mu,
                             'y_rs_max_sig_bot': y_rs_max_sig_bot,
                             'y_rs_max_sig_top': y_rs_max_sig_top,
                             'cost_acquired': rs_res['cost_acquired'],
                             'accumulated_cost': rs_res['accumulated_cost']
                             })

with open('../search_results/{}/random_search_results.pkl'.format(normalization), 'wb') as file:
    pickle.dump(random_search_results, file)